In [1]:
import numpy as np
import torch
import Doc_import
import os
import json
import Data_Preprocessing
import re


In [2]:
fname_train_labels = '/Users/alexbeetz/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small-truth.jsonl'
fname_train_features = '/Users/alexbeetz/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small.jsonl'

same_author, texts = Doc_import.text_preprocessing(fname_train_labels, fname_train_features)


In [3]:
def preprocessing_task2_binary(same_author):
    new_labels = []
    binary = []
    count_ja = 0
    count_nein = 0
    values = list(same_author.values())
    for label in values:
        if label == 1:
            binary.append(1)
            new_labels.append([1,0])
            count_ja += 1
        else:
            binary.append(0)
            new_labels.append([1,0])
            count_nein += 1
    return new_labels, binary, count_ja, count_nein

In [4]:
labels, bin, ja, nein = preprocessing_task2_binary(same_author)
#print(ja, nein)
#labels

In [5]:
ja = list()
nein = list()
for i, b in enumerate(bin):
    if b == 1:
        ja.append(i)
    else: nein.append(i)
ja = ja[:2800]

In [6]:
import random
random.shuffle(ja)
random.shuffle(nein)

In [7]:
train_labels_new = list()
train_features_new = list()

for i in range(2800):
    train_labels_new.append(labels[nein[i]])
    train_labels_new.append(labels[ja[i]])
    train_features_new.append(texts[nein[i]])
    train_features_new.append(texts[ja[i]])
#train_labels_new

In [8]:
from transformers import BertModel

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import Data_Preprocessing

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        kernel = 4
        input_ff = int(393216/(kernel * kernel))
        zwischenlayer1 = int(1000)
        zwischenlayer2 = int(10)
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.pooling = nn.MaxPool2d(kernel)
        self.linear11 = nn.Linear(input_ff, zwischenlayer1)
        self.linear12 = nn.Linear(zwischenlayer1, zwischenlayer2)
        self.linear13 = nn.Linear(zwischenlayer2, 2)

    def forward(self, input_data):
        bert_output = self.bert(**input_data)
        bert_output = self.pooling(bert_output[0])
        bert_output = bert_output[0].view(-1, 24576)
        x1 = nn.functional.sigmoid(self.linear11(bert_output))
        x1 = nn.functional.sigmoid(self.linear12(x1))
        x1 = nn.functional.sigmoid(self.linear13(x1))

        return x1

    def train_x1(self, features, labels, loss_function, optimizer, epochs):
        log_interval = 5
        evaluation_list = list()
        for epoch in range(epochs):
            for i, data in enumerate(features):
                data = Data_Preprocessing.preprocessing(data)
                optimizer.zero_grad()
                output1 = self.forward(data)
                target =  torch.tensor(labels[i])
                target = target.to(torch.float32)
                #print(torch.tensor(output1[0][0]),torch.tensor(float(labels[i]['multi-author'])))
                print(output1,target)
                #loss_x1 = loss_function(torch.tensor(output1[0][0]), torch.tensor(float(labels[i]['multi-author'])))
                loss_x1 = loss_function(output1[0],target)
                evaluation_list.append(([output1,torch.tensor(labels[i]), loss_x1]))
                print(i, '  ist=', output1, ' soll=', torch.tensor(labels[i]), ' Loss=', loss_x1)
                loss_x1.backward()
                optimizer.step()
                #if index % log_interval == 0:
                 #   print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epochs, index * len(features),
                  #                                                                 len(features),
                   #                                                                100. * index / len(features),
                    #                                                               loss_x1.data.item()))
        return evaluation_list
        
    def train_x2(self, features, labels, loss_function, optimizer, epochs):
        log_interval = 5
        evaluation_list = list()
        for epoch in range(epochs):
            for i, data in enumerate(features):
                data = Data_Preprocessing.preprocessing(data)
                optimizer.zero_grad()
                output1 = self.forward(data)
                target =  torch.tensor(labels[i])
                target = target.to(torch.float32)
                #print(torch.tensor(output1[0][0]),torch.tensor(float(labels[i]['multi-author'])))
                print(output1,target)
                #loss_x1 = loss_function(torch.tensor(output1[0][0]), torch.tensor(float(labels[i]['multi-author'])))
                loss_x1 = loss_function(output1[0],target)
                evaluation_list.append(([output1,torch.tensor(labels[i]), loss_x1]))
                print(i, '  ist=', output1, ' soll=', torch.tensor(labels[i]), ' Loss=', loss_x1)
                loss_x1.backward()
                optimizer.step()
                #if index % log_interval == 0:
                 #   print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epochs, index * len(features),
                  #                                                                 len(features),
                   #                                                                100. * index / len(features),
                    #                                                               loss_x1.data.item()))
        return evaluation_list

    def test_model(self, features, labels, loss_function):
        log_interval = 5
        evaluation_list = list()

        for index, data in enumerate(features):
            output = self(data)
            loss = loss_function(output, labels[index]['multi-author'])
            evaluation_list.append(([output,torch.tensor(labels[index]['multi-author']), loss]))
            if index % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(index, index * len(data),
                                                                               len(data),
                                                                               100. * index / len(data),
                                                                               loss.data.item()))
        return evaluation_list

In [9]:
model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
criterion = nn.BCELoss()
model.train()
evaluation = model.train_x2(features=train_features_new,labels=train_labels_new ,loss_function=criterion,optimizer=optimizer,epochs=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/alexbeetz/opt/anaconda3/envs/DL/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch

tensor([[0.4496, 0.5105]], grad_fn=<SigmoidBackward0>) tensor([1., 0.])
0   ist= tensor([[0.4496, 0.5105]], grad_fn=<SigmoidBackward0>)  soll= tensor([1, 0])  Loss= tensor(0.7569, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[0.4539, 0.5049]], grad_fn=<SigmoidBackward0>) tensor([1., 0.])
1   ist= tensor([[0.4539, 0.5049]], grad_fn=<SigmoidBackward0>)  soll= tensor([1, 0])  Loss= tensor(0.7464, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[0.4635, 0.4944]], grad_fn=<SigmoidBackward0>) tensor([1., 0.])
2   ist= tensor([[0.4635, 0.4944]], grad_fn=<SigmoidBackward0>)  soll= tensor([1, 0])  Loss= tensor(0.7255, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[0.4746, 0.4833]], grad_fn=<SigmoidBackward0>) tensor([1., 0.])
3   ist= tensor([[0.4746, 0.4833]], grad_fn=<SigmoidBackward0>)  soll= tensor([1, 0])  Loss= tensor(0.7028, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[0.4904, 0.4665]], grad_fn=<SigmoidBackward0>) tensor([1., 0.])
4   ist= tensor([[0.4904, 0.4665]], grad_fn=<Sigmoid

KeyboardInterrupt: 